<a href="https://colab.research.google.com/github/ljkrajewski/jupyter_notebooks/blob/main/book_writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to write a book
## Outline
~~Write an outline for a three-act story in the {genre} genre. For each act, write a list of chapters along with descriptions/summaries for each chapter.~~<br>
_Done._

## Individual chapters
~~For each chapter:~~
- ~~write a list of 3-5 sections within the chapter~~
- ~~summaries/descriptions for each section.~~

_Done._

## Chapter sections
For each chapter section:
- Write the section. Include:
  - Story summaries so far
  - List of characters and facts about them
  - List of other story-related facts
- Write a summary of the section (in a bullet-point format)
- Write/add to a list of characters along with facts about those characters (in a bullet-point format)
- Write/add to a list of story-related facts

In [ ]:
#@title Defined globals
model_name="llama3" #@param {type: "string"}  The name of the LLM.
debug=True #@param {type: "boolean"}
model_endpoint="http://localhost:11434/api/generate" #The endpoint for the LLM's API.

import requests
import json

##Install and start ollama, then pull the model.

In [ ]:
#@title Install ollama
!curl -fsSL https://ollama.com/install.sh | sh


In [ ]:
#@title Start ollama
import multiprocessing
import os
import time

def run_ollama():
    print(f"Running ollama on PID {os.getpid()}")
    os.system('ollama serve')

ollama_process = multiprocessing.Process(target=run_ollama)
ollama_process.start()
time.sleep(10)

In [ ]:
!ollama pull $model_name

In [ ]:
#@title Test ollama connection
!curl http://localhost:11434

In [ ]:
#@title Defined functions
# prompt: Write a function that takes a dictionary prompt and sends a request to an LLM's API. The output is given in dictionary.

def query_llm(prompt):
  global model_endpoint,model_name

  headers = {
      "Content-Type": "application/json",
  }
  data = {
      "model": model_name,
      "prompt": prompt,
      "stream": False
  }

  answer = requests.post(model_endpoint, headers=headers, json=data)

  if answer.status_code == 200:
    try:
      answer_dict = json.loads(answer.content)
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
      return None
    return answer_dict["response"].replace("\n", " ")
  else:
    print(f"Error: {answer.status_code}")
    return None


## Write the book

In [ ]:
#@title Create the outline and chapters
# prompt: Pick a random literary genre. Send a request to an LLM to write an outline for a three-act story in the selected genre. For each act, write a list of chapters along with descriptions/summaries for each chapter.

import random

genres = ["fantasy", "sci-fi", "mystery", "romance", "horror", "thriller", "historical fiction", "western", "dystopian"]
random_genre = random.choice(genres)
print(f"Random genre: {random_genre}")
print("")

prompt = """
Write an outline for a three-act story in the {random_genre} genre. For each act, write a list of chapters along with descriptions/summaries for each chapter. Provide your answer in the following JSON format:
{ "act1": { "chapter1": { "description": "Description of chapter 1", "subsections": [] }, "chapter2": { "description": "Description of chapter 2", "subsections": [] }, ... }, "act2": { "chapter6": { "description": "Description of chapter 6", "subsections": [] }, "chapter7": { "description": "Description of chapter 7", "subsections": [] }, ... }, ... }
Do not provide any additional information outside of the JSON result.
"""

response = query_llm(prompt) + " }"
try:
  story = json.loads(response)
except json.JSONDecodeError as e:
  print(f"Error decoding JSON: {e}")
  story = None
if debug:
  print(json.dumps(story,indent=2))

In [ ]:
#@title Add sections to chapters
# prompt: Break up each chapter in dictionary "story" into 3 to 5 sub-sections that fit the chapter summary and add them as a list under their respective chapters

def break_chapter_into_sections(chapter_summary):
  """Breaks a chapter summary into 3-5 sub-sections."""

  prompt = f"""
  Break the following chapter summary into 3-5 sub-sections, each section progessing the story within that chapter.
  Provide your answer in a list the following JSON format:
  ["section 1 description", "section 2 description", ...]
  Do not forget the commas between each section description. Do not title the sections. Do not provide any additional information outside of the JSON result.

  Chapter Summary: {chapter_summary}
  """

  response = query_llm(prompt)
  return response

for act in story.keys():
  if debug:
    print("==> Act: "+str(act))
  for chapter in story[act].keys():
    if debug:
      print("====> Chapter: "+str(chapter))
      print("      "+story[act][chapter]['description'])
    sections_text = break_chapter_into_sections(story[act][chapter]['description'])
    if sections_text[-1] != ']':
      sections_text += " ]"
    story[act][chapter]['subsections'] = json.loads(sections_text)
    for i in range(len(story[act][chapter]['subsections'])):
      if debug:
        print("--> "+str(i))
        print(story[act][chapter]['subsections'][i])
        print("")